# Seminario 5:  Python Mágico

## Implementación de la clase Matriz

In [37]:
class Matriz:
    def __init__(self, filas, columnas, valor_inicial=0):
        self.filas = filas
        self.columnas = columnas
        self.matriz = [[valor_inicial]*columnas for _ in range(filas)]

    def __getitem__(self, indices):
        fila, columna = indices
        return self.matriz[fila][columna]

    def __setitem__(self, indices, valor):
        fila, columna = indices
        self.matriz[fila][columna] = valor
        
    def __getattr__(self, attr):
        fila, columna = map(int, attr.split('_')[1:])
        return self.matriz[fila][columna]

    def __setattr__(self, attr, valor):
        if attr.startswith('_'):
            fila, columna = map(int, attr.split('_')[1:])
            self.matriz[fila][columna] = valor
        else:
            # super().__setattr__(attr, valor)
            object.__setattr__(self, attr, valor)

    def __iter__(self):
        for fila in self.matriz:
            for valor in fila:
                yield valor

    def as_type(self, tipo):
        nueva_matriz = Matriz(self.filas, self.columnas)
        nueva_matriz.matriz = [[tipo(valor) for valor in fila] for fila in self.matriz]
        return nueva_matriz

    def __str__(self):
        return '\n'.join([' '.join(map(str, fila)) for fila in self.matriz])

    # def _1_1 (self):
    #     print("se vuelve loco?")

    def __add__(self, other):
        if self.filas != other.filas or self.columnas != other.columnas:
            raise ValueError('Las matrices deben tener las mismas dimensiones')
        nueva_matriz = Matriz(self.filas, self.columnas)
        nueva_matriz.matriz = [[self[fila, columna] + other[fila, columna] for columna in range(self.columnas)] for fila in range(self.filas)]
        return nueva_matriz
    
    def __sub__(self, other):
        if self.filas != other.filas or self.columnas != other.columnas:
            raise ValueError('Las matrices deben tener las mismas dimensiones')
        nueva_matriz = Matriz(self.filas, self.columnas)
        nueva_matriz.matriz = [[self[fila, columna] - other[fila, columna] for columna in range(self.columnas)] for fila in range(self.filas)]
        return nueva_matriz
    
    def __mul__(self, other):
        if self.columnas != other.filas:
            raise ValueError('El número de columnas de la primera matriz debe ser igual al número de filas de la segunda matriz')
        nueva_matriz = Matriz(self.filas, other.columnas)
        nueva_matriz.matriz = [[sum(self[fila, k] * other[k, columna] for k in range(self.columnas)) for columna in range(other.columnas)] for fila in range(self.filas)]
        return nueva_matriz


In [38]:
m = Matriz(2, 2, 2)
n = Matriz(2, 2)
n.matriz = [[1, 2], [3, 4]]

print(m*n)

8 12
8 12


// estoy leyendome esto

La resolución de métodos y miembros en Python se refiere a cómo Python busca y encuentra métodos y atributos en una clase y sus superclases. Aquí hay algunos recursos donde puedes aprender más sobre este tema:

1. Documentación oficial de Python sobre el modelo de datos: https://docs.python.org/3/reference/datamodel.html. Esta página contiene información sobre cómo Python busca métodos especiales en las clases y sus superclases.

2. Documentación oficial de Python sobre la función `super()`: https://docs.python.org/3/library/functions.html#super. Esta página explica cómo puedes usar `super()` para llamar a métodos en la superclase de una clase.

3. Tutorial de Python sobre herencia múltiple: https://docs.python.org/3/tutorial/classes.html#multiple-inheritance. Esta sección del tutorial de Python explica cómo Python resuelve los métodos y atributos cuando una clase hereda de múltiples superclases.

4. Artículo de Real Python sobre el método de resolución de orden (MRO) en Python: https://realpython.com/python-super/. Este artículo explica en detalle cómo Python determina el orden en el que busca métodos y atributos en las superclases de una clase.

Estos recursos deberían ayudarte a entender cómo Python resuelve métodos y miembros en las clases y sus superclases.